In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1IFTVQVhXDAoHIk3Ewmz5AKLUo0lFt1N5", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/00_intro.mp3"))

In [ ]:
#@title 🎧 Listen: Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

# Vision Encoders & Patch Embeddings: How VLAs "See" the Road

*Part 1 of the Vizuara series on Vision-Language-Action Models for Autonomous Driving*
*Estimated time: 60 minutes | Runs on: Google Colab (T4 GPU)*

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://pods.vizuara.ai/courses/vla-autonomous-driving/practice/1/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


## 1. Why Does This Matter?

Let us start with a simple question: how does a self-driving car *see*?

A human driver glances at the road ahead, checks the mirrors, notices a pedestrian stepping off the curb, spots a red brake light two cars ahead — all in a fraction of a second. We process rich visual information effortlessly. But for an autonomous vehicle, the raw camera feed is just a grid of numbers — millions of pixel values streaming in from 4 to 8 cameras with 360-degree coverage.

**Vision-Language-Action models (VLAs)** are a new class of models that take these camera images, convert them into compact "feature tokens" (just like words in a sentence), feed them to a language model, and output driving actions — steering, acceleration, braking. The first and arguably most critical step in this entire pipeline is the **Vision Encoder**: the component that converts raw pixels into meaningful representations.

The dominant approach? The **Vision Transformer (ViT)**. It splits each camera image into a grid of small patches, treats each patch as a "word," and uses self-attention to understand how the patches relate to each other.

By the end of this notebook, you will:
- Build a complete Vision Transformer from scratch in PyTorch
- Train it on real images and achieve reasonable accuracy
- Visualize **attention heatmaps** showing exactly which parts of the image the model focuses on
- Understand every mathematical operation happening inside

Let us get started.

In [ ]:
#@title 🎧 Listen: Building Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_building_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 2. Building Intuition

Before we touch any code or equations, let us build a strong mental model for what is happening.

**Analogy 1: Reading a newspaper page.** Imagine you pick up a newspaper. You do not read every single letter from top-left to bottom-right. Your eyes jump to headlines, photographs, bold text, and key paragraphs. You quickly understand the "layout" — where the important stuff is. A Vision Transformer does something remarkably similar with images. It divides the image into patches and learns which patches matter and how they relate to each other.

**Analogy 2: Tiles in a mosaic.** Think of a large mosaic on a wall. Each small tile contains a tiny piece of the picture — maybe a sliver of blue sky, or a fragment of a face. Individually, each tile tells you very little. But your brain naturally groups tiles together: "these blue tiles are sky," "these tan tiles form a face," "these red tiles are a flower." The transformer's self-attention mechanism does the same thing — it figures out which tiles (patches) are related. In driving, this means: "this patch has a red light AND this patch shows a car bumper ahead, so there is a car braking in front of us."

### Think About This

Why patches instead of individual pixels? This comes down to computational cost.

Consider a typical driving camera image: 320 x 576 pixels. That is 184,320 pixels. Self-attention computes relationships between *every pair* of tokens, which is O(n^2). So:
- **Pixel-level attention:** 184,320^2 = ~34 billion operations. Not feasible.
- **With 16x16 patches:** we get (320/16) x (576/16) = 20 x 36 = 720 patches. And 720^2 = 518,400 operations. Entirely manageable.

Patches give us a 65,000x reduction in the number of attention computations. This is exactly what makes Vision Transformers practical.

In [ ]:
#@title 🎧 Listen: Mathematics
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_mathematics.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 3. The Mathematics

Now let us look at the three key mathematical operations inside a Vision Transformer. We will take each one, explain what it does computationally, and work through a concrete numerical example.

### 3.1 Patch Embedding

$$\mathbf{z}_i = \mathbf{W}_p \cdot \text{flatten}(\text{patch}_i) + \mathbf{b}_p$$

What does this say? Take each 16 x 16 x 3 patch (that is 16 x 16 pixels across 3 color channels = 768 values), flatten it into a single vector, and multiply by a learned weight matrix $\mathbf{W}_p$ to project it into a $d$-dimensional embedding space. Then add a bias term. This is literally a linear projection — a matrix multiplication followed by an addition.

**Let us plug in some simple numbers.** Suppose we have a tiny 4 x 4 x 1 patch (grayscale, so 1 channel). Flattened, this gives us a vector of 16 values. Say our embedding dimension is $d = 3$:

$$\text{patch} = [0.2, 0.5, 0.1, 0.8, \ldots] \quad (\text{16 values})$$

$$\mathbf{W}_p \in \mathbb{R}^{3 \times 16}, \quad \mathbf{b}_p \in \mathbb{R}^{3}$$

$$\mathbf{z}_i = \mathbf{W}_p \cdot \text{patch} + \mathbf{b}_p = [1.3, -0.7, 0.4]$$

So a 16-dimensional raw patch gets compressed into a 3-dimensional embedding vector. In practice, the embedding dimension is much larger (e.g., 128 or 768), but the operation is exactly the same — a matrix multiply.

### 3.2 Positional Encoding

$$\mathbf{z}_i' = \mathbf{z}_i + \mathbf{e}_i^{\text{pos}}$$

After embedding each patch, we add a **learnable position vector** so the model knows *where* each patch came from in the original image. Without positional encoding, the model would see a bag of patches with no spatial information — it could not distinguish "car in the top-left" from "car in the bottom-right."

**Worked example.** Suppose patch 0 (top-left corner) has embedding $\mathbf{z}_0 = [1.3, -0.7, 0.4]$ and its learnable position vector is $\mathbf{e}_0^{\text{pos}} = [0.1, 0.2, -0.1]$:

$$\mathbf{z}_0' = [1.3 + 0.1, \; -0.7 + 0.2, \; 0.4 + (-0.1)] = [1.4, -0.5, 0.3]$$

Patch 5 (somewhere in the middle) would have a different position vector $\mathbf{e}_5^{\text{pos}} = [-0.3, 0.5, 0.6]$, so even if two patches had identical pixel content, their final representations would differ based on where they sit in the image. This is exactly what we want.

### 3.3 Self-Attention

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

This is the heart of the transformer. Let us break it down computationally:

- **Q (Query):** Each patch asks "what am I looking for?"
- **K (Key):** Each patch declares "what do I contain?"
- **$QK^T$ (dot product):** Measures compatibility between every pair of patches. High score means "these two patches are relevant to each other."
- **$\sqrt{d_k}$ (scaling):** Prevents the dot products from growing too large, which would push softmax into regions where gradients vanish.
- **Softmax:** Converts raw scores into attention weights that sum to 1 — a probability distribution over patches.
- **Multiply by V (Value):** Produces a weighted combination of all patches, emphasizing the most relevant ones.

**Worked example with 3 patches, $d_k = 2$.** Suppose:

$$Q = \begin{bmatrix} 1 & 0 \\ 0 & 1 \\ 1 & 1 \end{bmatrix}, \quad K = \begin{bmatrix} 1 & 1 \\ 0 & 1 \\ 1 & 0 \end{bmatrix}, \quad V = \begin{bmatrix} 0.5 & 0.3 \\ 0.8 & 0.1 \\ 0.2 & 0.9 \end{bmatrix}$$

Step 1 — Compute $QK^T$:

$$QK^T = \begin{bmatrix} 1 & 0 & 1 \\ 1 & 1 & 0 \\ 2 & 1 & 1 \end{bmatrix}$$

Step 2 — Scale by $\sqrt{d_k} = \sqrt{2} \approx 1.41$:

$$\frac{QK^T}{\sqrt{2}} = \begin{bmatrix} 0.71 & 0 & 0.71 \\ 0.71 & 0.71 & 0 \\ 1.41 & 0.71 & 0.71 \end{bmatrix}$$

Step 3 — Apply softmax to each row (we get attention weights):

Row 1: $[0.40, 0.20, 0.40]$ — Patch 1 attends equally to patches 1 and 3, less to patch 2.

Step 4 — Multiply by V: the output for patch 1 would be $0.40 \times [0.5, 0.3] + 0.20 \times [0.8, 0.1] + 0.40 \times [0.2, 0.9] = [0.44, 0.50]$.

This tells us that patch 1's new representation is a weighted blend of all patches, with more weight on patches 1 and 3. In a driving scene, this is how the model learns that a brake light patch and a bumper patch should attend to each other.

In [ ]:
#@title 🎧 Listen: Start Building
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_start_building.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 4. Let Us Build It — Component by Component

Now we have all the intuition and math. Let us implement each component in PyTorch, step by step, and visualize what each one does.

### Setup

In [ ]:
# Setup — run this first!
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms
%matplotlib inline

torch.manual_seed(42)
np.random.seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

In [ ]:
#@title 🎧 Listen: Patch Embedding
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/06_patch_embedding.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.1 Patch Embedding

The first step is splitting the image into patches and projecting each one into an embedding vector. There is a neat trick here: a `Conv2d` layer with `kernel_size = stride = patch_size` is *mathematically equivalent* to extracting non-overlapping patches and applying a linear projection to each. Let us implement this.

In [ ]:
class PatchEmbedding(nn.Module):
    """Split an image into non-overlapping patches and project to embedding dimension.

    A Conv2d with kernel_size=stride=patch_size extracts patches and applies
    a linear projection in one operation. This is equivalent to:
      1. Slice the image into patch_size x patch_size blocks
      2. Flatten each block to a vector of length (channels * patch_size^2)
      3. Multiply by a weight matrix W of shape (embed_dim, patch_dim)
    """
    def __init__(self, img_size=32, patch_size=4, in_channels=3, embed_dim=128):
        super().__init__()
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2
        # Conv2d trick: kernel_size=stride=patch_size
        self.projection = nn.Conv2d(
            in_channels, embed_dim,
            kernel_size=patch_size, stride=patch_size
        )

    def forward(self, x):
        # x: (batch, channels, height, width)
        x = self.projection(x)    # (batch, embed_dim, n_patches_h, n_patches_w)
        x = x.flatten(2)          # (batch, embed_dim, n_patches)
        x = x.transpose(1, 2)     # (batch, n_patches, embed_dim)
        return x

# Quick test
patch_embed = PatchEmbedding()
dummy_img = torch.randn(1, 3, 32, 32)
out = patch_embed(dummy_img)
print(f"Input shape:  {dummy_img.shape}")
print(f"Output shape: {out.shape}")
print(f"Number of patches: {patch_embed.n_patches}")
print(f"Each patch is now a {out.shape[-1]}-dimensional vector")

In [ ]:
#@title 🎧 Listen: Patch Visualization
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/07_patch_visualization.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Now let us actually *see* what patch extraction looks like on a real image.

In [ ]:
# Visualize patch extraction on a CIFAR-10 image
transform = transforms.Compose([transforms.ToTensor()])
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
sample_img, label = dataset[0]
cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                 'dog', 'frog', 'horse', 'ship', 'truck']

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Original image
axes[0].imshow(sample_img.permute(1, 2, 0).numpy())
axes[0].set_title(f'Original Image (32x32) — "{cifar_classes[label]}"', fontsize=12)
axes[0].axis('off')

# Image with patch grid overlay
img_with_grid = sample_img.permute(1, 2, 0).numpy().copy()
axes[1].imshow(img_with_grid)
patch_size = 4
for i in range(0, 32, patch_size):
    axes[1].axhline(y=i, color='red', linewidth=0.8)
    axes[1].axvline(x=i, color='red', linewidth=0.8)
axes[1].set_title(f'Divided into {(32//patch_size)**2} patches (4x4 each)', fontsize=12)
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Show all 64 individual patches in a grid
patch_size = 4
img_np = sample_img.permute(1, 2, 0).numpy()
n_patches_per_side = 32 // patch_size  # 8

fig, axes = plt.subplots(n_patches_per_side, n_patches_per_side, figsize=(6, 6))
for i in range(n_patches_per_side):
    for j in range(n_patches_per_side):
        patch = img_np[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size]
        axes[i][j].imshow(patch)
        axes[i][j].axis('off')
plt.suptitle(
    f'Image split into {n_patches_per_side}x{n_patches_per_side} = {n_patches_per_side**2} patches\n'
    f'Each patch: {patch_size}x{patch_size} pixels = {patch_size*patch_size*3} values (RGB)',
    fontsize=12
)
plt.tight_layout()
plt.show()

print(f"Each patch has {patch_size}x{patch_size}x3 = {patch_size*patch_size*3} raw values")
print(f"After projection, each patch becomes a 128-dimensional embedding vector")
print(f"So we go from {32*32*3:,} pixel values to {n_patches_per_side**2} x 128 = {n_patches_per_side**2 * 128:,} values")

Notice how each individual patch is tiny and hard to interpret on its own — just like a single mosaic tile. The transformer's job is to learn how these tiles fit together.

In [ ]:
#@title 🎧 Listen: Positional Encoding
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/08_positional_encoding.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.2 Positional Encoding

Now the question is: once we have patch embeddings, how does the model know *where* each patch came from? If we just pass a bag of patch vectors, the model has no idea whether a patch is from the top-left corner (sky) or bottom-center (road). We solve this by adding **learnable positional embeddings**.

We also introduce a special **[CLS] token** — an extra learnable vector prepended to the sequence. This token has no corresponding image patch. Instead, it learns to aggregate information from all patches through attention, and its final representation is used for classification.

In [ ]:
class PositionalEncoding(nn.Module):
    """Learnable positional encoding for patch tokens, plus a CLS token."""
    def __init__(self, n_patches, embed_dim):
        super().__init__()
        # CLS token: a learnable vector for classification
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        # One learnable position embedding per token (patches + CLS)
        self.position_embeddings = nn.Parameter(
            torch.randn(1, n_patches + 1, embed_dim)
        )

    def forward(self, x):
        batch_size = x.size(0)
        # Expand CLS token for the entire batch
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        # Prepend CLS token to the patch sequence
        x = torch.cat([cls_tokens, x], dim=1)  # (batch, n_patches+1, embed_dim)
        # Add positional encoding
        x = x + self.position_embeddings
        return x

# Quick test
pos_enc = PositionalEncoding(n_patches=64, embed_dim=128)
dummy_patches = torch.randn(2, 64, 128)  # batch of 2, 64 patches, 128-dim
out = pos_enc(dummy_patches)
print(f"Input:  {dummy_patches.shape} (batch, patches, embed_dim)")
print(f"Output: {out.shape} (batch, patches+CLS, embed_dim)")

Let us visualize the positional encoding similarity matrix. Before training, the position vectors are random. After training, we would expect nearby patches to have similar position encodings — because neighboring patches tend to contain related visual content.

In [ ]:
# Visualize positional encoding similarity
pos_enc_viz = PositionalEncoding(n_patches=64, embed_dim=128)
pos_embeds = pos_enc_viz.position_embeddings.squeeze(0).detach()

# Compute cosine similarity between all position pairs
similarity = F.cosine_similarity(
    pos_embeds.unsqueeze(0), pos_embeds.unsqueeze(1), dim=-1
)

plt.figure(figsize=(8, 6))
plt.imshow(similarity.numpy(), cmap='viridis')
plt.colorbar(label='Cosine Similarity')
plt.title('Positional Encoding Similarity (before training)\n'
          'Position 0 = CLS token, Positions 1-64 = image patches')
plt.xlabel('Token Position')
plt.ylabel('Token Position')
plt.show()

print("Before training, the similarity is essentially random noise.")
print("After training, we would see a structured pattern: nearby patches")
print("would have high similarity, and distant patches would have low similarity.")
print("The CLS token (position 0) would show moderate similarity to all patches.")

This is exactly what we want — a mechanism for the model to understand spatial layout.

In [ ]:
#@title 🎧 Listen: Multi Head Attention
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/09_multi_head_attention.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.3 Multi-Head Self-Attention

Now we arrive at the core mechanism. Self-attention lets every patch "look at" every other patch and decide which ones are relevant. Multi-head attention does this multiple times in parallel, with each "head" potentially learning to focus on different types of relationships.

Why multiple heads? Think of it this way: one head might learn to focus on color relationships ("these patches are all blue sky"), another on edges ("these patches form a continuous lane marking"), and another on semantic groupings ("these patches all belong to the same vehicle"). Having multiple heads gives the model richer, more diverse representations.

In [ ]:
class MultiHeadAttention(nn.Module):
    """Multi-head self-attention mechanism.

    Projects input to Q, K, V for each head, computes scaled dot-product
    attention, concatenates heads, and projects back.
    """
    def __init__(self, embed_dim, n_heads):
        super().__init__()
        self.n_heads = n_heads
        self.head_dim = embed_dim // n_heads
        assert embed_dim % n_heads == 0, "embed_dim must be divisible by n_heads"

        # Single linear layer to compute Q, K, V for all heads at once
        self.qkv = nn.Linear(embed_dim, 3 * embed_dim)
        self.projection = nn.Linear(embed_dim, embed_dim)
        self.attn_weights = None  # Store for visualization later

    def forward(self, x):
        batch_size, seq_len, embed_dim = x.shape

        # Compute Q, K, V for all heads in one matrix multiplication
        qkv = self.qkv(x)  # (batch, seq_len, 3 * embed_dim)
        qkv = qkv.reshape(batch_size, seq_len, 3, self.n_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # (3, batch, heads, seq_len, head_dim)
        q, k, v = qkv[0], qkv[1], qkv[2]

        # Scaled dot-product attention
        scale = self.head_dim ** 0.5
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / scale
        attn_weights = F.softmax(attn_scores, dim=-1)
        self.attn_weights = attn_weights.detach()  # Save for visualization

        # Apply attention weights to values
        attn_output = torch.matmul(attn_weights, v)

        # Concatenate heads and apply final projection
        attn_output = attn_output.transpose(1, 2).reshape(batch_size, seq_len, embed_dim)
        output = self.projection(attn_output)
        return output

# Quick test
mha = MultiHeadAttention(embed_dim=128, n_heads=4)
dummy_input = torch.randn(1, 65, 128)  # 64 patches + 1 CLS token
out = mha(dummy_input)
print(f"Input:  {dummy_input.shape}")
print(f"Output: {out.shape}")
print(f"Attention weights shape: {mha.attn_weights.shape}")
print(f"  -> {mha.attn_weights.shape[1]} heads, each computing {mha.attn_weights.shape[2]}x{mha.attn_weights.shape[3]} attention matrix")

In [ ]:
#@title 🎧 Listen: Transformer Block
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/10_transformer_block.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.4 Transformer Block

A single transformer block combines self-attention with a feed-forward network (FFN), plus residual connections and layer normalization. The residual connections are critical — they allow gradients to flow directly through the network, making it possible to stack many layers.

We use the **pre-norm** architecture (LayerNorm before attention/FFN), which is the standard in modern Vision Transformers. It tends to train more stably than post-norm.

In [ ]:
class TransformerBlock(nn.Module):
    """A single transformer encoder block: LayerNorm + Attention + Residual + LayerNorm + FFN + Residual."""
    def __init__(self, embed_dim, n_heads, mlp_ratio=4.0, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attention = MultiHeadAttention(embed_dim, n_heads)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, int(embed_dim * mlp_ratio)),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(int(embed_dim * mlp_ratio), embed_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        # Pre-norm: normalize before attention and FFN
        x = x + self.attention(self.norm1(x))  # Residual + Attention
        x = x + self.mlp(self.norm2(x))        # Residual + FFN
        return x

# Quick test
block = TransformerBlock(embed_dim=128, n_heads=4)
out = block(dummy_input)
print(f"Input:  {dummy_input.shape}")
print(f"Output: {out.shape} (same shape — transformer preserves dimensions)")

In [ ]:
#@title 🎧 Listen: Complete Vit
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/11_complete_vit.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.5 Complete Vision Transformer

Now we have all the pieces. Let us assemble the full ViT: Patch Embedding, Positional Encoding, a stack of Transformer Blocks, and a classification head.

In [ ]:
class SimpleViT(nn.Module):
    """A simple Vision Transformer for image classification.

    Architecture:
        Image -> PatchEmbedding -> PositionalEncoding -> N x TransformerBlock
              -> LayerNorm -> CLS token -> Linear classifier -> logits
    """
    def __init__(self, img_size=32, patch_size=4, in_channels=3,
                 n_classes=10, embed_dim=128, n_heads=4, n_layers=4, dropout=0.1):
        super().__init__()
        self.patch_embed = PatchEmbedding(img_size, patch_size, in_channels, embed_dim)
        n_patches = self.patch_embed.n_patches
        self.pos_encoding = PositionalEncoding(n_patches, embed_dim)
        self.dropout = nn.Dropout(dropout)

        # Stack of transformer blocks
        self.transformer = nn.Sequential(
            *[TransformerBlock(embed_dim, n_heads, dropout=dropout)
              for _ in range(n_layers)]
        )

        self.norm = nn.LayerNorm(embed_dim)
        self.classifier = nn.Linear(embed_dim, n_classes)

    def forward(self, x):
        x = self.patch_embed(x)       # (batch, n_patches, embed_dim)
        x = self.pos_encoding(x)      # (batch, n_patches+1, embed_dim) — CLS prepended
        x = self.dropout(x)
        x = self.transformer(x)       # (batch, n_patches+1, embed_dim)
        x = self.norm(x)
        cls_output = x[:, 0]          # Take CLS token output
        logits = self.classifier(cls_output)  # (batch, n_classes)
        return logits

# Create model and inspect
model = SimpleViT().to(device)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters:     {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"\nArchitecture summary:")
print(f"  Patch size: 4x4")
print(f"  Number of patches: 64")
print(f"  Embedding dim: 128")
print(f"  Attention heads: 4")
print(f"  Transformer layers: 4")
print(f"  Sequence length: 65 (64 patches + 1 CLS token)")

Not bad — we have a complete Vision Transformer in under 100 lines of code. In a real autonomous driving VLA like RT-2 or OpenVLA, this same architecture (scaled up significantly) processes each camera image and produces the feature tokens that the language model reads. The concept is identical.

In [ ]:
#@title 🎧 Listen: Todo1
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/12_todo1.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 5. Your Turn

Now it is your turn to implement two core operations from scratch. These exercises will cement your understanding of what is happening inside the ViT.

### TODO 1: Implement Scaled Dot-Product Attention from Scratch

We saw the attention formula earlier: $\text{softmax}(QK^T / \sqrt{d_k}) \cdot V$. Now implement it yourself. No peeking at the `MultiHeadAttention` class above!

In [ ]:
def scaled_dot_product_attention(Q, K, V):
    """
    Compute scaled dot-product attention from scratch.

    Args:
        Q: Query tensor of shape (batch, heads, seq_len, head_dim)
        K: Key tensor of shape (batch, heads, seq_len, head_dim)
        V: Value tensor of shape (batch, heads, seq_len, head_dim)

    Returns:
        output: (batch, heads, seq_len, head_dim)
        attn_weights: (batch, heads, seq_len, seq_len)
    """
    d_k = Q.size(-1)

    # ============ TODO ============
    # Step 1: Compute attention scores: Q @ K^T (transpose last two dims of K)
    # Step 2: Scale by sqrt(d_k)
    # Step 3: Apply softmax over the last dimension (each query attends to all keys)
    # Step 4: Multiply attention weights by V to get weighted values
    # ==============================

    scores = ???       # YOUR CODE HERE — shape: (batch, heads, seq_len, seq_len)
    attn_weights = ??? # YOUR CODE HERE — shape: (batch, heads, seq_len, seq_len)
    output = ???       # YOUR CODE HERE — shape: (batch, heads, seq_len, head_dim)

    return output, attn_weights

In [ ]:
# Verification cell — run this to check your implementation
Q_test = torch.randn(1, 2, 4, 8)
K_test = torch.randn(1, 2, 4, 8)
V_test = torch.randn(1, 2, 4, 8)

out, weights = scaled_dot_product_attention(Q_test, K_test, V_test)
assert out.shape == (1, 2, 4, 8), f"Wrong output shape: {out.shape}, expected (1, 2, 4, 8)"
assert weights.shape == (1, 2, 4, 4), f"Wrong attention shape: {weights.shape}, expected (1, 2, 4, 4)"
assert torch.allclose(weights.sum(dim=-1), torch.ones(1, 2, 4), atol=1e-5), \
    "Attention weights don't sum to 1 along the last dimension"
print("Your attention implementation is correct!")
print(f"  Output shape: {out.shape}")
print(f"  Attention weights shape: {weights.shape}")
print(f"  Weights sum per query: {weights.sum(dim=-1)[0, 0]} (should be all 1.0)")

In [ ]:
#@title 🎧 Listen: Todo2
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/13_todo2.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### TODO 2: Implement Patch Embedding Without the Conv2d Trick

In our `PatchEmbedding` class, we used a Conv2d shortcut. But what is it actually doing? Let us implement it the explicit way: manually extract patches, flatten them, and apply a linear projection.

In [ ]:
def manual_patch_embed(image, patch_size, projection_weight, projection_bias):
    """
    Manually extract patches and project them — no Conv2d shortcut.

    Args:
        image: (batch, channels, height, width)
        patch_size: int — size of each square patch
        projection_weight: (embed_dim, patch_dim) where patch_dim = channels * patch_size^2
        projection_bias: (embed_dim,)

    Returns:
        patch_embeddings: (batch, n_patches, embed_dim)
    """
    batch, channels, height, width = image.shape
    n_patches_h = height // patch_size
    n_patches_w = width // patch_size

    # ============ TODO ============
    # Step 1: Extract patches using unfold or manual slicing
    #         Hint: image.unfold(dim, size, step) extracts sliding windows
    #         You need to unfold along both height (dim=2) and width (dim=3)
    #         Then reshape so each patch is flattened to (channels * patch_size^2,)
    # Step 2: Apply linear projection: patches @ weight^T + bias
    # ==============================

    patches = ???       # YOUR CODE HERE — shape: (batch, n_patches, patch_dim)
    embeddings = ???    # YOUR CODE HERE — shape: (batch, n_patches, embed_dim)

    return embeddings

In [ ]:
# Verification cell
test_img = torch.randn(2, 3, 32, 32)
embed_dim = 128
patch_size = 4
patch_dim = 3 * patch_size * patch_size  # 48
W = torch.randn(embed_dim, patch_dim)
b = torch.randn(embed_dim)

result = manual_patch_embed(test_img, patch_size=patch_size, projection_weight=W, projection_bias=b)
assert result.shape == (2, 64, 128), f"Wrong shape: {result.shape}, expected (2, 64, 128)"

# Bonus: verify it gives same result as Conv2d
conv = nn.Conv2d(3, embed_dim, kernel_size=patch_size, stride=patch_size, bias=True)
with torch.no_grad():
    conv.weight.copy_(W.reshape(embed_dim, 3, patch_size, patch_size))
    conv.bias.copy_(b)
conv_result = conv(test_img).flatten(2).transpose(1, 2)
manual_result = manual_patch_embed(test_img, patch_size, W, b)
match = torch.allclose(conv_result, manual_result, atol=1e-5)
print(f"Manual patch embedding works! Shape: {result.shape}")
print(f"Matches Conv2d output: {match}")
if match:
    print("You now understand exactly what Conv2d does under the hood.")

In [ ]:
#@title 🎧 Listen: Training
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/14_training.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 6. Training on CIFAR-10

Enough building — let us train our Vision Transformer and see how it performs. We will use CIFAR-10: 60,000 tiny 32x32 images across 10 classes (airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, trucks). It is small enough to train quickly on a T4 GPU but complex enough to be a meaningful test.

In [ ]:
# Prepare data with augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True
)

print(f"Training samples: {len(train_dataset):,}")
print(f"Test samples:     {len(test_dataset):,}")
print(f"Classes: {cifar_classes}")
print(f"Batches per epoch: {len(train_loader)}")

In [ ]:
# Training loop — should take ~5 minutes on a T4 GPU
model = SimpleViT(
    img_size=32, patch_size=4, in_channels=3,
    n_classes=10, embed_dim=128, n_heads=4, n_layers=4
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
criterion = nn.CrossEntropyLoss()

train_losses = []
test_accs = []

print("Training SimpleViT on CIFAR-10...")
print("-" * 50)

for epoch in range(10):
    # --- Training ---
    model.train()
    epoch_loss = 0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        logits = model(images)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = logits.max(1)
        correct_train += predicted.eq(labels).sum().item()
        total_train += labels.size(0)

    avg_loss = epoch_loss / len(train_loader)
    train_acc = 100. * correct_train / total_train
    train_losses.append(avg_loss)
    scheduler.step()

    # --- Evaluation ---
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            _, predicted = logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    acc = 100. * correct / total
    test_accs.append(acc)
    print(f"Epoch {epoch+1:2d}/10 | Loss: {avg_loss:.4f} | Train Acc: {train_acc:.1f}% | Test Acc: {acc:.1f}%")

print("-" * 50)
print(f"Final test accuracy: {test_accs[-1]:.1f}%")

For a small ViT trained from scratch on only 50,000 images with no pretraining, getting around 70-75% accuracy is reasonable. State-of-the-art ViTs pretrained on ImageNet-21k and fine-tuned on CIFAR-10 achieve over 99%, but they have orders of magnitude more parameters and training data. The important thing is that our model *learns* — the attention mechanism works.

In [ ]:
#@title 🎧 Listen: Training Results
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/15_training_results.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# Training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(range(1, 11), train_losses, 'b-o', linewidth=2, markersize=6)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Training Loss', fontsize=12)
ax1.set_title('Training Loss over Epochs', fontsize=13)
ax1.grid(True, alpha=0.3)

ax2.plot(range(1, 11), test_accs, 'r-o', linewidth=2, markersize=6)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Test Accuracy (%)', fontsize=12)
ax2.set_title('Test Accuracy over Epochs', fontsize=13)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Best test accuracy: {max(test_accs):.1f}% (epoch {test_accs.index(max(test_accs))+1})")

This is exactly what we want — the loss decreases steadily and accuracy improves. Our Vision Transformer is learning to classify images by attending to informative patches.

In [ ]:
#@title 🎧 Listen: Attention Heatmaps
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/16_attention_heatmaps.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 7. Final Output: Attention Heatmaps

This is the most exciting part. Let us look *inside* the trained model and see where it focuses its attention. We will extract the attention weights from the last transformer layer and visualize which patches the CLS token attends to. This tells us which parts of the image the model considers most important for classification.

In a real driving VLA, these same attention patterns would reveal which parts of the road scene the model focuses on — brake lights, lane markings, pedestrians, traffic signs.

In [ ]:
# Extract attention maps from the trained model
model.eval()

# Get a batch of test images
dataiter = iter(test_loader)
test_images, test_labels = next(dataiter)
test_images = test_images.to(device)

# Register forward hooks to capture attention weights from each layer
attention_maps = []
def hook_fn(module, input, output):
    if hasattr(module, 'attn_weights') and module.attn_weights is not None:
        attention_maps.append(module.attn_weights.cpu())

hooks = []
for block in model.transformer:
    hooks.append(block.attention.register_forward_hook(hook_fn))

# Forward pass to trigger hooks
with torch.no_grad():
    logits = model(test_images)
    predictions = logits.argmax(dim=-1).cpu()

# Clean up hooks
for h in hooks:
    h.remove()

print(f"Captured attention maps from {len(attention_maps)} transformer layers")
print(f"Each attention map shape: {attention_maps[0].shape}")
print(f"  -> (batch={attention_maps[0].shape[0]}, heads={attention_maps[0].shape[1]}, "
      f"seq_len={attention_maps[0].shape[2]}, seq_len={attention_maps[0].shape[3]})")

In [ ]:
# Visualize attention heatmaps for 4 sample images
from PIL import Image

# Unnormalize images for display
unnorm = transforms.Normalize(
    mean=[-0.4914/0.2470, -0.4822/0.2435, -0.4465/0.2616],
    std=[1/0.2470, 1/0.2435, 1/0.2616]
)

fig, axes = plt.subplots(4, 3, figsize=(12, 14))
column_titles = ['Input Image', 'Attention Heatmap\n(CLS token, last layer)', 'Attention Overlay']

for col, title in enumerate(column_titles):
    axes[0, col].set_title(title, fontsize=12, fontweight='bold')

for idx in range(4):
    # --- Column 1: Original image ---
    img = unnorm(test_images[idx].cpu()).clamp(0, 1)
    axes[idx, 0].imshow(img.permute(1, 2, 0).numpy())
    pred_class = cifar_classes[predictions[idx]]
    true_class = cifar_classes[test_labels[idx]]
    color = 'green' if pred_class == true_class else 'red'
    axes[idx, 0].set_xlabel(f'True: {true_class} | Pred: {pred_class}',
                            fontsize=10, color=color)
    axes[idx, 0].set_xticks([])
    axes[idx, 0].set_yticks([])

    # --- Column 2: Attention heatmap ---
    # Last layer attention: CLS token (position 0) attending to all patches (positions 1+)
    last_attn = attention_maps[-1][idx]  # (heads, seq_len, seq_len)
    cls_attn = last_attn[:, 0, 1:]       # (heads, n_patches) — CLS attending to patches
    avg_attn = cls_attn.mean(0)           # Average across heads -> (n_patches,)

    # Reshape to 2D spatial grid
    n_patches_side = int(avg_attn.shape[0] ** 0.5)
    attn_map = avg_attn.reshape(n_patches_side, n_patches_side).numpy()

    axes[idx, 1].imshow(attn_map, cmap='hot', interpolation='bilinear')
    axes[idx, 1].set_xticks([])
    axes[idx, 1].set_yticks([])

    # --- Column 3: Overlay ---
    img_np = img.permute(1, 2, 0).numpy()
    # Resize attention map to image size for overlay
    attn_resized = np.array(
        Image.fromarray((attn_map * 255 / attn_map.max()).astype(np.uint8)).resize(
            (32, 32), Image.BILINEAR
        )
    ) / 255.0

    axes[idx, 2].imshow(img_np)
    axes[idx, 2].imshow(attn_resized, cmap='hot', alpha=0.5)
    axes[idx, 2].set_xticks([])
    axes[idx, 2].set_yticks([])

plt.suptitle('What Does the Vision Transformer Focus On?', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

print("The bright/hot regions show where the model pays the most attention.")
print("Notice how it tends to focus on the object itself rather than the background.")
print()
print("In a real driving VLA, these attention patterns would highlight:")
print("  - Other vehicles (especially brake lights)")
print("  - Lane markings and road boundaries")
print("  - Traffic signs and signals")
print("  - Pedestrians and cyclists")

In [ ]:
# Bonus: compare attention patterns across different heads
fig, axes = plt.subplots(2, 4, figsize=(14, 6))

sample_idx = 0
img = unnorm(test_images[sample_idx].cpu()).clamp(0, 1)

# Show original image
axes[0, 0].imshow(img.permute(1, 2, 0).numpy())
axes[0, 0].set_title(f'Original\n({cifar_classes[test_labels[sample_idx]]})', fontsize=10)
axes[0, 0].axis('off')

# Show each head's attention from the last layer
last_attn = attention_maps[-1][sample_idx]  # (heads, seq_len, seq_len)
n_heads = last_attn.shape[0]

for head in range(n_heads):
    row = (head + 1) // 4
    col = (head + 1) % 4

    head_attn = last_attn[head, 0, 1:]  # CLS attention to patches
    n_side = int(head_attn.shape[0] ** 0.5)
    attn_grid = head_attn.reshape(n_side, n_side).numpy()

    axes[row, col].imshow(attn_grid, cmap='hot', interpolation='bilinear')
    axes[row, col].set_title(f'Head {head + 1}', fontsize=10)
    axes[row, col].axis('off')

# Clear unused subplots
for i in range(n_heads + 1, 8):
    row = i // 4
    col = i % 4
    axes[row, col].axis('off')

plt.suptitle('Different Attention Heads Focus on Different Things', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print("Each head learns a different attention pattern:")
print("  Some heads might focus on edges and contours")
print("  Others might attend to color regions or textures")
print("  This diversity is exactly why multi-head attention is powerful")

This is truly amazing. With just a few hundred lines of code, we have built a model that learns to "see" — it automatically discovers which parts of an image are important, just like a human driver scanning the road.

In [ ]:
#@title 🎧 Listen: Connecting To Driving
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/17_connecting_to_driving.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 8. Connecting to Autonomous Driving VLAs

Let us take a step back and connect what we built to how real autonomous driving VLAs work.

In a production VLA like RT-2, OpenVLA, or DriveVLM:

1. **Multiple cameras** (4-8) capture 360-degree views around the vehicle
2. **Each camera image** is processed by a Vision Transformer (much larger than ours — typically ViT-Large or ViT-Huge with ~300M-600M parameters)
3. The ViT produces **feature tokens** — one per patch, exactly like our model
4. These tokens are **concatenated** across all cameras, giving the language model a complete visual representation of the driving scene
5. The language model processes these visual tokens alongside text instructions (e.g., "turn left at the next intersection") and outputs **driving actions** (steering angle, acceleration, braking)

The patch embedding and attention mechanisms we implemented are the exact same operations running in these production systems. The only differences are scale (more parameters, higher resolution images, larger patches) and the downstream components (language model + action head instead of a simple classifier).

In [ ]:
# Scale comparison: our toy ViT vs. real driving ViTs
print("=" * 60)
print("Scale Comparison: Our ViT vs. Production Driving ViTs")
print("=" * 60)
print(f"{'':20s} {'Ours':>15s} {'ViT-Large':>15s} {'ViT-Huge':>15s}")
print("-" * 60)
print(f"{'Image size':20s} {'32x32':>15s} {'224x224':>15s} {'336x336':>15s}")
print(f"{'Patch size':20s} {'4x4':>15s} {'16x16':>15s} {'14x14':>15s}")
print(f"{'Num patches':20s} {'64':>15s} {'196':>15s} {'576':>15s}")
print(f"{'Embed dim':20s} {'128':>15s} {'1024':>15s} {'1280':>15s}")
print(f"{'Heads':20s} {'4':>15s} {'16':>15s} {'16':>15s}")
print(f"{'Layers':20s} {'4':>15s} {'24':>15s} {'32':>15s}")
print(f"{'Parameters':20s} {f'{total_params:,}':>15s} {'304M':>15s} {'632M':>15s}")
print(f"{'Cameras':20s} {'1':>15s} {'6-8':>15s} {'6-8':>15s}")
print("-" * 60)
print("\nThe concepts are identical — only the scale changes.")
print("You now understand the core of how VLAs process visual input.")

In [ ]:
#@title 🎧 Listen: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/18_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 9. Reflection and Next Steps

### Reflection Questions

1. **Patch size tradeoff:** We used 4x4 patches on 32x32 images (64 patches). What would change if we used 2x2 patches (256 patches)? We would get finer spatial resolution — each patch captures more localized details. But the attention computation grows quadratically: 256^2 = 65,536 vs. 64^2 = 4,096. That is a 16x increase in attention cost. There is a fundamental tradeoff between resolution and computational cost.

2. **Why CLS token?** Why use a special CLS token instead of, say, averaging all patch embeddings? In the first transformer layer, before the model has learned anything useful, averaging random patch representations would give a noisy mess. The CLS token starts as a learnable "query" that progressively accumulates information from all patches through attention. It acts as a dedicated summarization slot. That said, some recent ViT variants do use global average pooling instead of CLS, and they work well too.

3. **Driving scene attention patterns:** Would you expect the attention pattern for a highway scene to look different from a city intersection? Absolutely. On a highway, the model would likely attend strongly to the lane markings, the car directly ahead, and the road curvature. At a city intersection, attention would be much more distributed — pedestrians, traffic lights, crossing vehicles, turn lanes. This is the beauty of self-attention: it adapts dynamically to the scene.

### Optional Challenges

1. **Larger patches:** Change `patch_size` to 8 (giving only 16 patches) and retrain. How does accuracy change? What do the attention maps look like with so few patches?

2. **More heads:** Try 8 heads instead of 4. Visualize each head's attention pattern separately — do different heads learn to focus on different visual features?

3. **Multi-camera fusion:** This is closer to real driving VLAs. Modify the model to process 4 images simultaneously (simulating front, left, right, and rear cameras). Concatenate their patch tokens into a single sequence before the transformer layers. Does the attention mechanism learn cross-camera relationships?

In [ ]:
# Quick starter for Challenge 1
print("Challenge 1: Larger patches")
print("Change patch_size from 4 to 8:")
print("  - Patches: 64 -> 16")
print("  - Each patch covers: 4x4=16 pixels -> 8x8=64 pixels")
print("  - Attention matrix: 65x65 -> 17x17 (much smaller)")
print()
print("Try it: model_8 = SimpleViT(patch_size=8).to(device)")
print("Then retrain and compare accuracy + attention maps")

That is it for Part 1. You have built a complete Vision Transformer from scratch, trained it, and visualized its attention patterns. You now understand the first and most critical component of how VLAs "see" the road.

In production autonomous driving systems, everything we built here runs at scale — larger images, more cameras, bigger models — but the fundamental operations are identical: split images into patches, embed them, add positions, run self-attention, and produce feature tokens that downstream models can understand.

Thanks for working through this notebook!